## Analysis of Yelp Dataset Portfolio2 

The portfolio aims to use various data processing techniques to clean and preprocess the data, filter out irrelevant data, and remove any outliers that could impact the accuracy of the model. Finally, the portfolio aims to use linear regression models to train and validate the model's accuracy in predicting user ratings towards business. Ultimately, the objective is to build a reliable model that can be used to provide insights into user preferences and help businesses make data-driven decisions to improve their services and offerings. In this task, we will explore the impacts of feature selections and different sizes of training/testing data on the model performance. We will use Yelp sub-dataset.

### Import the output sample data from Portfolio1
The csv file named `Yelp_Portfolio2_Input.csv` is provided. You may need to use the Pandas method, i.e., `read_csv`, for doing this. After that, please import the csv file and print out its total length.

In [1]:
# Name: Dylan Neilson
# Student ID: 47004029

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

# Import library to calculate MSE for Q3
from sklearn.metrics import mean_squared_error

In [3]:
# Import datasheet
sample_data = pd.read_csv('data/Yelp_Portfolio2_Input.csv')

# Print length on datasheet
print("The length of the datasheet: ", len(sample_data))

The length of the datasheet:  38654


# Q1. `business_categories` processing

You may notice that there are a total of 693 unique business categories present in the sample data. Please write a code snippet to confirm this number. Then, generate a boxplot of `stars` for `business_categories`.

In [4]:
# Confirm there are 693 unique business categories
category_array = sample_data['business_categories'].unique()
print("The number of unique business categories: ", category_array.size)

The number of unique business categories:  693


In [ ]:
# Generates a boxplot of stars for business_categories
sample_data.boxplot("stars",by = "business_categories")

<AxesSubplot:title={'center':'stars'}, xlabel='business_categories'>

Sometimes we need to extract some major categories, such as `Middle Eastern; Restaurants` and `Sushi Bars; Restaurants`, both of which should belong to the `Restaurants` category.

### Q1.1 How to efficiently and accurately extract the last category value (i.e., categorieN) from the `business_categories` column in a dataset, where the format of each row is a semicolon-separated list of categories (i.e., 'categorie1; categorie2;...;categorieN')? and replace the original string with this last category value. At last, print the first five rows.

In [ ]:
# Loop through each row in sample_data
for i in range(len(sample_data)): 
    # Get the categories for current row
    categories = sample_data.at[i, 'business_categories']
    # If semicolon exists, split the categories by semicolon
    if ';' in categories:
        split_categories = categories.split(';')
        # Get the last category stored in split_categories and remove any spaces
        last_category = split_categories[-1].split()[-1]
        # Overwrite the category value for this row with last_category
        sample_data.at[i, 'business_categories'] = last_category
    else: 
        # If semicolon does not exist, remove the spaces in the string
        sample_data.at[i, 'business_categories'] = categories.split()[-1]

# Display first 5 rows of dataframe
sample_data.head()

### Q1.2 Let's first check how many unique business categories are present in the dataset (output from Q1.1). Maybe it's still not an appropriate number to make a clear Boxplot. Now, we want to display `ALL Rows` of the business category count data. That is to count and print the number of occurrences for each business category.

In [ ]:
# Check the amount of unique business categories in the cleaned data from Q1.1
category_array = sample_data['business_categories'].unique()
print("The number of unique business categories: ", category_array.size)

In [ ]:
# Store the number of occurances for each business category
business_cat_count = sample_data['business_categories'].value_counts()

# Display all rows of DataFrame
pd.options.display.max_rows = None

# Print the number of occurances for each business category
print(business_cat_count)

### Q1.3 We want to remove categories that appear fewer than 200 times. Write a Python code snippet to perform this operation. At last, print the number of resulting unique business_categories.

In [ ]:
# Store the number of occurances for each business category
business_cat_count = sample_data['business_categories'].value_counts()

# Store the number of businesses that appear more than 200 times
business_count = business_cat_count[business_cat_count >= 200].index

# Check each element in the 'business_categories' column sample_data is present in the business_count list
sample_data = sample_data[sample_data['business_categories'].isin(business_count)]

# Print the result
print("business categories that occur more than 200 times: ", len(sample_data))

# Display all rows of dataframe
pd.options.display.max_rows = None

# Print the occurance of individual businesses
print(business_cat_count)

# Q2. Outliers

### Q2.1 Remove any rows from the sample_data DataFrame (output from Q1.3) where the `useful` column has a value of 6 or more; Then, create a single figure with two subplots, one showing the boxplot of `useful` column of the original data and the other showing the boxplot of the data with outliers removed. 

At last, __print the length of the data__.

In [ ]:
# Store rows from useful column that has a value less than 6 (This removes any values >6)
useful_less_six_df = sample_data[sample_data['useful'] < 6]

# Create a single figure with two subplots
fig, axs = plt.subplots(1, 2, figsize = (10, 5))

# Create first subplot for the orginal data
axs[0].boxplot(sample_data['useful'])
axs[0].set_title('Original Data')

# Create second subplot for the cleaned data
axs[1].boxplot(useful_less_six_df['useful'])
axs[1].set_title('Data After Removing Values of 6 or more')

# Show the figure
plt.show()

In [ ]:
# Print the result
print("Length of cleaned data: ", len(useful_less_six_df))

### Q2.2 In general, we might remove some inactive users from a dataset (output from Q2.1), for example, users who rate businesses less than 3 times. However, in this case, we are doing the opposite and removing extremely active users who rate businesses more than 30 times (>30). Again, create a single figure with two subplots, one showing the boxplot of the count of business rating given by each user and the other showing the boxplot of the data with outliers removed. 

At last, __print the length of the data__.

In [ ]:
# Group and count the number of reviews by each user in useful_less_six_df
user_group_df = useful_less_six_df.groupby(['user_id'])['stars'].count().reset_index(name = "count")

# Store number of users who rate the business 30 times or less (This removes any values >30)
userIDs = user_group_df[user_group_df['count'] <= 30]['user_id'].tolist()

# Store rows from useful column that only include the users that exist in userIDs
userid_outliers_removed_df = useful_less_six_df[useful_less_six_df['user_id'].isin(userIDs)]

# Create a single figure with two subplots
fig, axs = plt.subplots(1,2,figsize=(10,5))

# Create first subplot for the orginal data
axs[0].boxplot(useful_less_six_df['useful'])
axs[0].set_title('Data Before Removing Users')

# Create second subplot for the cleaned data
axs[1].boxplot(userid_outliers_removed_df['useful'])
axs[1].set_title('Data After removing Users')

# Show the figure
plt.show()

In [ ]:
# Print the result
print("Length of cleaned data: ", len(userid_outliers_removed_df))

# Q3. Training a LinearRegression model on the data output from Q2.2.

### Q3.1 How to build a linear regression model on a subset of the data, using the `useful` column as the input variable and the `stars` column as the output variable. Split the data into a training set and a test set, with 70% of the data used for training. The random_state is set to 42.

In [ ]:
# Define X and y input
X = userid_outliers_removed_df[['useful']]
y = userid_outliers_removed_df['stars']

# Split dataset into training and testing sets, 30% for testing and 70% for training, fixed random seed of 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create linear regression object
reg = linear_model.LinearRegression()
# Train regression model reg on the training set (X_train and y_train).
reg.fit(X_train, y_train)

# Make predicitons on the test set
y_pred = reg.predict(X_test)

# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE on the test set
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print MSE and RMSE
print('MSE is: ', mse)
print('RMSE is: ', rmse)

### Q3.2 Assuming that the number of reviews a business has received  `business_review_count`  can greatly influence the average rating that the business has received. Now, we create the relevant variable to the data, and do the LinearRegression again to the `stars`. 

In [ ]:
# Group rows in DataFrame by business ID, count number of reviews for each business, save the results in review_counts.
review_counts = userid_outliers_removed_df.groupby('business_id')['text'].count().reset_index()
# Renames 'text' column in the review_counts DataFrame
review_counts = review_counts.rename(columns={'text': "business_review_count"})
# Display first 5 rows of review_counts DataFrame
review_counts.head()

In [ ]:
# Create new DataFrame and merge userid_outliers_removed_df DataFrame with review_counts DataFrame on 'business_id' column
new_data = pd.merge(userid_outliers_removed_df, review_counts, on='business_id')

# Define X and y input
X = new_data[['business_review_count']]
y = new_data['stars']
# Split dataset into training and testing sets, 30% for testing and 70% for training, fixed random seed of 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create linear regression object
reg = linear_model.LinearRegression()
# Train regression model reg on the training set (X_train and y_train).
reg.fit(X_train, y_train)

# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE on the test set
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print MSE and RMSE
print('MSE is: ', mse)
print('RMSE is: ', rmse)

### Q3.3 Output the correlations between `business_review_count` and `useful` variables with `stars` and analyze the results.

In [ ]:
# Find correlation coefficient between 'business_review_count' column and the 'stars' column
new_data['business_review_count'].corr(new_data['stars'])
# Find the correlation coefficient between 'useful' column and 'stars' column
new_data['useful'].corr(new_data['stars'])

# Print the correlation coefficient between the 'business_review_count' column and the 'stars' column
print("Correlations between bussiness_review_count and stars: ", new_data['business_review_count'].corr(new_data['stars']))
# Print the correlation coefficient between the 'useful' column and 'stars' column
print("Correlations between useful and stars: ", new_data['useful'].corr(new_data['stars']))

In [ ]:
# My Analysis

# If the correlation coefficient is positive, it indicates a positive relationship between the input and output 
# variables, and if it is negative, it indicates a negative relationship. A larger coefficient value indicates 
# a stronger correlation, while a smaller value indicates a weaker correlation, as in the case of the correlation 
# between 'useful' and 'stars'.